In [1]:
import pandas as pd
import time
from better_profanity import profanity
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date

In [2]:
reddit_2020_csv = ['D:/reddit_data/raw_data/2020-01.csv',
                   'D:/reddit_data/raw_data/2020-02.csv',
                   'D:/reddit_data/raw_data/2020-03.csv',
                   'D:/reddit_data/raw_data/2020-04.csv',
                   'D:/reddit_data/raw_data/2020-05.csv',
                   'D:/reddit_data/raw_data/2020-06.csv',
                   'D:/reddit_data/raw_data/2020-07.csv',
                   'D:/reddit_data/raw_data/2020-08.csv',
                   'D:/reddit_data/raw_data/2020-09.csv',
                   'D:/reddit_data/raw_data/2020-10.csv',
                   'D:/reddit_data/raw_data/2020-11.csv',
                   'D:/reddit_data/raw_data/2020-12.csv']

reddit_2021_csv = ['D:/reddit_data/raw_data/2021-01.csv',
                   'D:/reddit_data/raw_data/2021-02.csv',
                   'D:/reddit_data/raw_data/2021-03.csv',
                   'D:/reddit_data/raw_data/2021-04.csv',
                   'D:/reddit_data/raw_data/2021-05.csv',
                   'D:/reddit_data/raw_data/2021-06.csv']

In [3]:
def create_df(csv_file, search = None, start_date = None, end_date = None):
    # read in file
    df = pd.read_csv(csv_file, low_memory=False)
    # filter to columns of interest
    df = df[['author', 'created_utc', 'body', 'score']]
    # update date format from epoch
    df['created_utc'] = pd.to_datetime(df['created_utc'],  errors='coerce', unit='s')
    # drop any columns that are all N/A
    df1 = df.dropna(how='all', axis=1)
    # drop any rows for comments that were deleted
    df2 = df1[~df1.body.str.contains('[deleted]', na=False) | ~df1.body.str.contains('[removed]', na=False)]
    # drop any rows with negative scores
    df2 = df2.drop(df2[(df2.score <= 0)].index)
    # filter to comments related to stock of interest
    if search is not None:
        df3 = df2[df2['body'].str.contains(search, na=False)]
    else:
        df3 = df2
    
    # if user entered a date range
    # this is still WIP. can't get the dates to compare correctly yet
    if start_date is not None and end_date is not None:
        # update date format
        df3['created_utc'] = pd.to_datetime(df3['created_utc']).dt.date
        # filter by date range
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        df4 = df3.loc[(df3['created_utc'] >= start_date) | (df3['created_utc'] <= end_date)]
    else:
        df4 = df3
    
#     # duplicate comment column for censoring
#     df4['body_clean'] = df4['body'] 
#     #censor the comment
#         for idx, row in df4.iterrows():
#             text = df4.loc[idx,'body_clean']
#             df4.loc[idx,'body_clean'] = profanity.censor(text)
#     # print dataframe
    return df4

# if using a google drive url (must be unzipped)
# url = 'https://drive.google.com/file/d/1LnJXQR4-W03y1RFliBSZzOIVsQcy_osF/view?usp=sharing'
# url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
# search should be formatted like 'pepsi|Pepsi|PEP'
# date should be formatted YYYY/MM/DD
frames = []
for csv in reddit_2020_csv:
    final_df = create_df(csv)
    frames.append(final_df)

In [4]:
result = pd.concat(frames)
display(result)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


,author,created_utc,body,score
2,'optionseller',2020-01-01 00:00:49,'amazing',2
5,'connnor4real',2020-01-01 00:01:33,'U',10
16,'bobloadmire',2020-01-01 00:02:49,"""Isn't LTCG 15%?""",1
53,'grammerknewzi',2020-01-01 00:11:24,'AHAHAHAHAHAHHHAHH',1
78,'Howtoshortdow',2020-01-01 00:16:24,'No',11
...,...,...,...,...
1757827,'danielbauwens',2020-12-31 23:57:51,'My man.',4
1757842,'Healthynumbers',2020-12-31 23:58:18,'💎🤲',2
1757862,'STONK_MASTER21',2020-12-31 23:58:43,'Zoom',6
1757875,'DrugsnHugs69',2020-12-31 23:58:57,'Moron',1


In [5]:
result.to_csv('D:/reddit_data/df_2020.csv')